---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
from networkx.algorithms import bipartite
def answer_one():
        
    # Your Code Here
    G = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter="\t")
    
    return G # Your Answer Here
answer_one()

#!head -5 Employee_Movie_Choices.txt

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    
    # Your Code Here
    #G = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter="\t")
    
    B=nx.Graph()
    B.add_nodes_from(bipartite.sets(answer_one())[0],type='employee')
    B.add_nodes_from(bipartite.sets(answer_one())[1],type='movie')
    B.add_edges_from(answer_one().edges())
    #[G.set_node_attributes('e', type='employee') for e in bipartite.sets(G)[0]]
    #[G.set_node_attributes('e', type='movie') for e in bipartite.sets(G)[0]]
    return B# Your Answer Here
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
        
    # Your Code Here
    X=bipartite.sets(answer_one())[0]
    P=bipartite.weighted_projected_graph(answer_two(),X)
    
    return P# Your Answer Here
answer_three()
#plot_graph(answer_three(), weight_name=None)

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [6]:
def answer_four():
        
    # Your Code Here
    corr=pd.read_csv('Employee_Relationships.txt', delim_whitespace=True, 
                   header=None, names=['e1', 'e2', 'relation'])
    taste = pd.DataFrame(answer_three().edges(data=True),columns=['e1','e2','movie'])
    taste['movie']=taste['movie'].map(lambda x:x['weight'] if type(x)==dict else None)
    #it is very important that we have movie like in both directions since the relationship can
    #happen in both directions
    taste_copy=taste.copy()
    taste_copy.rename(columns={'e1':'e2_','e2':'e1'}, inplace=True)
    taste_copy.rename(columns={'e2_':'e2'}, inplace=True)
    taste_movie=pd.concat([taste,taste_copy])
    result = pd.merge(corr, taste_movie, on=['e1', 'e2'],how='left')
    #result=result.fillna(0)
    result[['relation','movie']] = result[['relation','movie']] .fillna(0)
    df=result.corr(method ='pearson')
    


    
    return df.iloc[1]['relation'] # Your Answer Here
answer_four()
#!head -5 Employee_Relationships.txt

0.78839622217334759